In [ ]:
import os
import cv2
import gdown
import shutil
import numpy as np
from PIL import Image
import numpy as np
from PIL import Image
from pathlib import Path
!pip install imagehash
from imagehash import average_hash

In [ ]:
import re

# Define the Google Drive URL
drive_url = "https://drive.google.com/file/d/13JlxoeE1CAo8zEwJKUrpDtaqO9gtyt-4/view?usp=sharing"

# Use regular expression to extract the file_id
file_id_match = re.search(r"/file/d/([^/]+)", drive_url)

if file_id_match:
    file_id = file_id_match.group(1)
    print(f"File ID: {file_id}")
else:
    print("File ID not found in the URL.")


File ID: 13JlxoeE1CAo8zEwJKUrpDtaqO9gtyt-4


In [ ]:
import os
import cv2
import gdown
import shutil
from PIL import Image
from imagehash import dhash

file_id = '13JlxoeE1CAo8zEwJKUrpDtaqO9gtyt-4'

# Define the URL for the shared file
url = f"https://drive.google.com/uc?id={file_id}"

# Specify the destination folder
output_folder = "/content/extracted_images"
os.makedirs(output_folder, exist_ok=True)

# Download the video file
output_video_file = os.path.join(output_folder, "video.mp4")
gdown.download(url, output_video_file, quiet=False)

# Create folders for extracted images, similar images, and split images
output_snapshots_folder = os.path.join(output_folder, "folder_1")
os.makedirs(output_snapshots_folder, exist_ok=True)
output_similar_images_folder = os.path.join(output_folder, "folder_2")
os.makedirs(output_similar_images_folder, exist_ok=True)

# Open the video file
cap = cv2.VideoCapture(output_video_file)
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_count = 0
previous_frame = None

# Function to check if two images are similar using dhash
def are_images_similar(image1, image2, threshold=10):
    hash1 = dhash(image1)
    hash2 = dhash(image2)
    return hash1 - hash2 <= threshold

while True:
    ret, frame = cap.read()

    if not ret:
        break

    if frame_count % fps == 0:  # Capture 1 frame per second
        # Save the frame as an image in folder_1
        image_path = os.path.join(output_snapshots_folder, f"frame_{frame_count//fps}.png")
        cv2.imwrite(image_path, frame)  # Save as PNG format

        # Check for similar images in folder_1 and copy only the first of the similar images
        if previous_frame:
            current_image = Image.open(image_path)
            previous_image = Image.open(previous_frame)
            if are_images_similar(current_image, previous_image, threshold=10):
                os.remove(image_path)
            else:
                previous_frame = image_path
        else:
            previous_frame = image_path

    frame_count += 1

# Move the remaining images to folder_2
for image_file in os.listdir(output_snapshots_folder):
    image_path = os.path.join(output_snapshots_folder, image_file)
    shutil.move(image_path, os.path.join(output_similar_images_folder, image_file))

cap.release()
cv2.destroyAllWindows()
print("Task completed.")


Downloading...
From: https://drive.google.com/uc?id=13JlxoeE1CAo8zEwJKUrpDtaqO9gtyt-4
To: /content/extracted_images/video.mp4
100%|██████████| 95.2M/95.2M [00:00<00:00, 126MB/s]


Task completed.
